In [1]:
#import some important library 
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
data = json.loads(open('intents.json').read())

In [3]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [4]:
print(words)

["'s", 'a', 'accept', 'anyone', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'delivery', 'do', 'doe', 'funny', 'get', 'good', 'goodbye', 'have', 'hello', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'only', 'pay', 'paypal', 'see', 'sell', 'shipping', 'something', 'take', 'tell', 'thank', 'thanks', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [5]:
print(classes)

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [6]:
print(doc_X)

['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", "Thank's a lot!", 'Which items do you have?', 'What kinds of items are there?', 'What do you sell?', 'Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?', 'How long does delivery take?', 'How long does shipping take?', 'When do I get my delivery?', 'Tell me a joke!', 'Tell me something funny!', 'Do you know a joke?']


In [7]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'items', 'items', 'items', 'payments', 'payments', 'payments', 'payments', 'delivery', 'delivery', 'delivery', 'funny', 'funny', 'funny']


In [8]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [9]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               7168      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 15,879
Trainable params: 15,879
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================] - 0s 5ms/step - loss: 2.0798 - accuracy: 0.1154
Epoch 2/200
1/1 

1/1 [==============================] - 0s 0s/step - loss: 7.6052e-04 - accuracy: 1.0000
Epoch 73/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 74/200
1/1 [==============================] - 0s 998us/step - loss: 6.6875e-04 - accuracy: 1.0000
Epoch 75/200
1/1 [==============================] - 0s 0s/step - loss: 0.0044 - accuracy: 1.0000
Epoch 76/200
1/1 [==============================] - 0s 998us/step - loss: 0.0011 - accuracy: 1.0000
Epoch 77/200
1/1 [==============================] - ETA: 0s - loss: 6.3404e-04 - accuracy: 1.00 - 0s 1000us/step - loss: 6.3404e-04 - accuracy: 1.0000
Epoch 78/200
1/1 [==============================] - 0s 0s/step - loss: 0.0228 - accuracy: 1.0000
Epoch 79/200
1/1 [==============================] - 0s 1000us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 80/200
1/1 [==============================] - 0s 1000us/step - loss: 6.4013e-04 - accuracy: 1.0000
Epoch 81/200
1/1 [==============================] - 0s 999u

1/1 [==============================] - 0s 0s/step - loss: 0.0015 - accuracy: 1.0000
Epoch 152/200
1/1 [==============================] - 0s 1000us/step - loss: 5.7438e-04 - accuracy: 1.0000
Epoch 153/200
1/1 [==============================] - 0s 999us/step - loss: 3.1259e-04 - accuracy: 1.0000
Epoch 154/200
1/1 [==============================] - 0s 1000us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 155/200
1/1 [==============================] - 0s 998us/step - loss: 2.7942e-04 - accuracy: 1.0000
Epoch 156/200
1/1 [==============================] - 0s 1ms/step - loss: 4.2920e-04 - accuracy: 1.0000
Epoch 157/200
1/1 [==============================] - 0s 999us/step - loss: 8.5148e-04 - accuracy: 1.0000
Epoch 158/200
1/1 [==============================] - 0s 999us/step - loss: 3.7174e-04 - accuracy: 1.0000
Epoch 159/200
1/1 [==============================] - 0s 0s/step - loss: 0.0048 - accuracy: 1.0000
Epoch 160/200
1/1 [==============================] - 0s 1ms/step - loss: 2.0339e-04 - a

In [10]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab): 
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json): 
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = random.choice(i["responses"])
        break
    return result

In [11]:
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    if (message ==result):
        print(result)
    else:
        print("Your output is not here")

hi
Your output is not here
goodbye


UnboundLocalError: local variable 'result' referenced before assignment